In [0]:
import pandas as pd
import numpy as np
import os


train_labels = pd.read_csv('train_label.csv')
train_data = pd.read_csv('train_data.csv')

In [0]:
train_labels.head()

In [0]:
train_df = pd.merge(train_data, train_labels, on="id")

In [0]:
train_df.head()

In [0]:
# for i in train_labels.Humour.unique():
#    os.mkdir("train_set/train_set/" + i)

In [0]:
# import shutil
# for i in train_labels.Humour.unique():
#    temp_df = list(train_labels.id[train_labels.Humour==i])
#    for y in temp_df:
#        shutil.move("train_set/train_set/" + y, "train_set/train_set/" + i + "/" + y)

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.05,
        height_shift_range=0.05,
        rescale=1./255,
        shear_range=0.05,
        zoom_range=0.05,
        fill_mode='nearest')

def aug_images(origin, destination, amount):
    # the names of the images
    img_strings = os.listdir('train_set/train_set/' + origin)
    # loop through the images
    for img_str in img_strings:
        # load PIL image
        img = load_img('train_set/train_set/' + origin + '/' + img_str)
        # check if it is PNG and convert to JPG
        if img.format == 'PNG':
            temp_img = img.convert('RGB')
            img = temp_img
        # this is a Numpy array with shape (3, 150, 150)
        x = img_to_array(img)  
        # this is a Numpy array with shape (1, 3, 150, 150)
        x = x.reshape((1,) + x.shape)  

        # the .flow() command below generates batches of randomly transformed images
        # and saves the results in the destination folder
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir='train_set/train_set/' + destination, save_prefix=img_str, save_format='jpeg'):
            i += 1
            if i >= amount:
                break
                


In [0]:
# what is the size of the image
import pandas as pd
img_strings = os.listdir('train_set/train_set/funny/')
img_dim = []
for img_str in img_strings:
    # load PIL image
    img = load_img('train_set/train_set/funny/' + img_str)
    img_dim.append([img.width, img.height])
    
df = pd.DataFrame(img_dim, columns={'width', 'height'})

export_csv = df.to_csv (r'C:\Users\Fabian\Documents\Studie\Master\Applied Machine Learning\Poster\export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

print(df)

In [0]:
df.std() + df.mean()

In [0]:
df.width.plot.hist()

In [0]:
df.mode()

In [0]:
# example of progressively loading images from file
from keras.preprocessing.image import ImageDataGenerator

dir_path = 'train_set/train_set/'
# numbers are based on mode, varified with histogram
img_width, img_height = 500, 400

# create generator
datagen = ImageDataGenerator(validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(dir_path, 
                                       target_size=(img_width, img_height),
                                       subset='training')
val_it = datagen.flow_from_directory(dir_path,
                                     target_size=(img_width, img_height),
                                     subset='validation')
# confirm the iterator works
batchX, batchy = train_it.next()
val_batchX, val_batchy  = val_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [0]:
val_batchy

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# First iteration of the image model --> simple CNN.
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, img_width, img_height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Now we flatten our input to 1 dimension for the fulle connected layers.
model.add(Flatten(input_shape=(img_width, img_height, 3)))  
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

In [0]:
model.fit_generator(
    train_it,
    epochs=50,
    validation_data=val_it
)